In [1]:
!which python

/Users/alelevier/Documents/github/ssd-pytorch/venv/bin/python


In [2]:
import enum
import math
import pdb
import platform
import time

import cv2
import numpy as np
import torch
import torch.nn.functional as F
from matplotlib import patches, patheffects
from matplotlib import pyplot as plt
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook

from ssdmultibox import utils
from ssdmultibox.bboxer import Bboxer, TensorBboxer
from ssdmultibox.config import cfg
from ssdmultibox.criterion import SSDLoss
from ssdmultibox.datasets import PascalDataset, TrainPascalFlatDataset, device
from ssdmultibox.models import SSDModel
from ssdmultibox.plotting import *
from ssdmultibox.predict import Predict
from ssdmultibox.utils import open_image

import matplotlib.pyplot as plt
plt.ion()

In [3]:
device

device(type='cpu')

In [4]:
losses = {'total':[], 'loc':[], 'conf':[]}

In [5]:
model = SSDModel().to(device)

In [6]:
LR = 1e-2

criterion = SSDLoss(alpha=.2)
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=0.0005)
current_time = time.time()

In [7]:
BATCH = 8

TrainPascalFlatDataset.__len__ = lambda self: BATCH

train_dataset = TrainPascalFlatDataset()

len(train_dataset) 

8

In [8]:
train_dataset

In [9]:
NUM_WORKERS = 0
SHUFFLE = False
    
dataloader = DataLoader(train_dataset, batch_size=BATCH, shuffle=SHUFFLE, num_workers=NUM_WORKERS)

vars(dataloader)

{'dataset': <ssdmultibox.datasets.TrainPascalFlatDataset at 0x1644bd978>,
 'batch_size': 8,
 'num_workers': 0,
 'collate_fn': <function torch.utils.data.dataloader.default_collate(batch)>,
 'pin_memory': False,
 'drop_last': False,
 'timeout': 0,
 'worker_init_fn': None,
 'sampler': <torch.utils.data.sampler.SequentialSampler at 0x1644bde10>,
 'batch_sampler': <torch.utils.data.sampler.BatchSampler at 0x1644bde48>,
 '_DataLoader__initialized': True}

In [10]:
EPOCHS = 1

for epoch in range(EPOCHS):
    steps = math.ceil(len(train_dataset) / BATCH)
    for step in tqdm_notebook(range(steps)):
        image_ids, ims, gt_bbs, gt_cats = next(iter(dataloader))

        # put data on device
        ims, gt_bbs, gt_cats = PascalDataset.to_device(ims, gt_bbs, gt_cats)

        # zero out gradients
        optimizer.zero_grad()

        # forward pass
        preds = model(ims)
        loss, loc_loss, conf_loss = criterion(preds, (gt_bbs, gt_cats))

        # backward pass
        loss.backward()
        optimizer.step()

        # stats
        if step % 5 == 0:
            print('epoch: {} step: {} loss: {:.4f} time: {:.4f}'.format(
                epoch, step, loss.item(), time.time() - current_time))
            current_time = time.time()
            losses['total'].append(loss.item())
            losses['loc'].append(loc_loss.item())
            losses['conf'].append(conf_loss.item())

pos_loss: 204.2087 neg_hard_mining_loss: 642.3280
n: 67.0 bbs_loss: 978.0898 cats_loss: 846.5367
epoch: 0 step: 0 loss: 27.2332 time: 14.9513



In [11]:
bbs_preds, cats_preds = preds

In [12]:
idx = 0
cls_id = 6
dataset = train_dataset

In [13]:
# targets
get_targets(gt_cats, idx)

(tensor([[100.0000, 100.0000, 200.0000, 200.0000],
         [ 50.0000, 100.0000, 250.0000, 200.0000],
         [100.0000, 100.0000, 300.0000, 200.0000],
         [ 97.2954, 100.0000, 202.7046, 200.0000]]), tensor([6, 6, 6, 6]))

In [14]:
get_targets(gt_cats, idx, bbs_preds)

(tensor([[100.4239,  99.7479, 200.3274, 199.8281],
         [ 49.5499,  99.1935, 250.1684, 200.5991],
         [100.4956,  99.8876, 299.5807, 200.4930],
         [ 96.9375,  99.4978, 202.0993, 199.5117]], grad_fn=<TakeBackward>),
 tensor([6, 6, 6, 6]))

single

In [15]:
# plot_anchor_bbs(dataset, image_ids, idx, gt_cats)

In [16]:
# plot_preds(dataset, image_ids, idx, bbs_preds, gt_cats)

In [17]:
# plot_nms_preds(dataset, image_ids, idx, preds, limit=5)

In [18]:
# plot_nms_single_preds(dataset, image_ids, idx, cls_id, preds, limit=5)

multiple

In [19]:
# plot_multiple(plot_anchor_bbs, plots=(2,2), dataset=dataset, image_ids=image_ids, gt_cats=gt_cats)

In [20]:
# plot_multiple(plot_preds, dataset=dataset, image_ids=image_ids, gt_cats=gt_cats, bbs_preds=bbs_preds)

In [21]:
# plot_multiple(plot_nms_preds, dataset=dataset, image_ids=image_ids, preds=preds)

In [22]:
# plot_multiple(plot_nms_single_preds, dataset=dataset, image_ids=image_ids, cls_id=cls_id, preds=preds)